### Feature engineering
- Objectives: 
    - This notebook conducts feature engineering to merge yield data with climate and NDVI data prior to modeling

In [1]:
# import modules
import pandas as pd

In [2]:
# Read yield, climate, and NDVI data
df_yield = pd.read_excel(r'C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\PROCESSED_DATA\YIELD\india_processed_yield.xlsx')
df_ndvi = pd.read_excel(r'C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\PROCESSED_DATA\NDVI\ndvi_rice_only.xlsx')
df_weather = pd.read_excel(r'C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\PROCESSED_DATA\WEATHER\india_districts_weather_data.xlsx')

#### Process yield data

In [3]:
# Drop Year column and rename Year End to Year
df_yield = df_yield.drop(columns=['Year'])
df_yield = df_yield.rename(columns={'Year End': 'year'})

# Rename Area (Hectare) to Area_ha, Production (Tonnes) to Production_t, and Yield (Tonnes/Hectare) to Yield_t_ha
df_yield = df_yield.rename(columns={'Area (Hectare)': 'Area_ha', 'Production (Tonnes)': 'Production_t', 'Yield (Tonnes/Hectare)': 'Yield_t_ha'})

In [4]:
df_yield

,GID_2,State,District,year,Season,Area_ha,Production_t,Yield_t_ha
0,IND.11.12_1,Gujarat,Gandhinagar,2001,Kharif,7600.0,17100.0,2.250000
1,IND.11.12_1,Gujarat,Gandhinagar,2002,Kharif,7000.0,17500.0,2.500000
2,IND.11.12_1,Gujarat,Gandhinagar,2003,Kharif,4600.0,9900.0,2.152174
3,IND.11.12_1,Gujarat,Gandhinagar,2004,Kharif,13100.0,35200.0,2.687023
4,IND.11.12_1,Gujarat,Gandhinagar,2005,Kharif,13200.0,29100.0,2.204545
...,...,...,...,...,...,...,...,...
5115,IND.7.9_1,Chhattisgarh,Dhamtari,2016,Kharif,173361.0,314275.0,1.812836
5116,IND.7.9_1,Chhattisgarh,Dhamtari,2017,Kharif,183504.0,603476.0,3.288626
5117,IND.7.9_1,Chhattisgarh,Dhamtari,2018,Kharif,144868.0,311175.0,2.147990
5118,IND.7.9_1,Chhattisgarh,Dhamtari,2019,Kharif,178763.0,433106.0,2.422794


#### Process climate data

In [5]:
# Extract year and month from time column in df_weather and query for years 2001 to 2020

# Extract years from datetime
df_weather['year'] = pd.DatetimeIndex(df_weather['time']).year 

# Extract months from dateime
df_weather['month'] = pd.DatetimeIndex(df_weather['time']).month

# Keep only relevant years and months
df_weather = df_weather.query('year >= 2001 and year <= 2020')
df_weather = df_weather.query('month >= 5 and month <= 11')

# give  the t2m and tp as rows for each month
df_weather_pivot = df_weather.pivot_table(index=['GID_2', 'year'], columns='month', values=['t2m', 'tp'])

# Update dataframe to give t2m_5, t2m_6, t2m_7, t2m_8, t2m_9, t2m_10, t2m_11, tp_5, tp_6, tp_7, tp_8, tp_9, tp_10, tp_11
df_weather_pivot.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_weather_pivot.columns]

# make df_weather_pivot a normal dataframe
df_weather_pivot = df_weather_pivot.reset_index()

In [6]:
df_weather_pivot

,GID_2,year,t2m_5,t2m_6,t2m_7,t2m_8,t2m_9,t2m_10,t2m_11,tp_5,tp_6,tp_7,tp_8,tp_9,tp_10,tp_11
0,IND.1.2_1,2001,27.779602,27.498596,27.159996,27.065147,26.963776,26.681686,26.782242,0.008971,0.008086,0.008806,0.011354,0.008078,0.009862,0.003568
1,IND.1.2_1,2002,28.117668,27.838570,27.757858,26.947472,26.789848,27.038239,26.910240,0.006937,0.006844,0.004978,0.008965,0.006565,0.007092,0.007167
2,IND.1.2_1,2003,28.308159,27.735573,26.816757,27.001274,26.774086,26.863770,27.651878,0.006019,0.007235,0.013133,0.010763,0.007493,0.008225,0.000710
3,IND.1.2_1,2004,27.483925,27.427406,27.078186,27.074654,26.950729,27.227112,27.423866,0.011527,0.011107,0.006972,0.010652,0.007157,0.006041,0.002525
4,IND.1.2_1,2005,28.218224,27.739922,27.368973,27.228470,27.032806,26.786316,26.760773,0.006475,0.014033,0.011885,0.006864,0.014474,0.008864,0.006348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12975,IND.8.1_1,2016,30.533081,29.719971,26.992584,26.943665,27.089325,27.075195,25.562042,0.000326,0.005848,0.014561,0.013302,0.011364,0.002862,0.000000
12976,IND.8.1_1,2017,30.328705,28.904694,27.065399,27.252380,28.153534,28.014404,25.397888,0.000252,0.009998,0.019982,0.014579,0.006303,0.002598,0.000010
12977,IND.8.1_1,2018,30.303711,29.388428,26.831696,26.647980,27.255646,29.066650,27.166504,0.000031,0.009285,0.020340,0.009168,0.001955,0.000294,0.000010
12978,IND.8.1_1,2019,29.514526,29.454742,27.665466,27.152374,27.054535,28.079620,26.826263,0.000032,0.007067,0.016162,0.019371,0.016436,0.003365,0.001031


#### Process NDVI data

In [7]:
# in df_ndvi Keep time, ndvi, gid_2 comumns
df_ndvi = df_ndvi[[ 'GID_2','time', '_1_km_monthly_NDVI']]

In [8]:
# Extract years from datetime
df_ndvi['year'] = pd.DatetimeIndex(df_ndvi['time']).year 

# Extract months from dateime
df_ndvi['month'] = pd.DatetimeIndex(df_ndvi['time']).month

# Keep only relevant years and months
df_ndvi = df_ndvi.query('year >= 2001 and year <= 2020')
df_ndvi = df_ndvi.query('month >= 5 and month <= 11')

# give  the t2m and tp as rows for each month
df_ndvi_pivot = df_ndvi.pivot_table(index=['GID_2', 'year'], columns='month', values='_1_km_monthly_NDVI')

# Reset index
df_ndvi_pivot = df_ndvi_pivot.reset_index()

# Rename the columns 5, 6, ... , 11 to NDVI_5, NDVI_6, ... , NDVI_11 
df_ndvi_pivot.columns = ['GID_2', 'year', 'NDVI_5', 'NDVI_6', 'NDVI_7', 'NDVI_8', 'NDVI_9', 'NDVI_10', 'NDVI_11']

In [10]:
# Join df_yield and df_weather_pivot
df_yield_weather = df_yield.merge(df_weather_pivot, on=['GID_2', 'year'], how='left')


In [12]:
# Join df_yield_weather and df_ndvi_pivot
df_yield_weather_ndvi = df_yield_weather.merge(df_ndvi_pivot, on=['GID_2', 'year'], how='left')

In [14]:
# Save output to excel
df_yield_weather_ndvi.to_excel(r'C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\PROCESSED_DATA\ANALYSIS_READY\india_yield_weather_ndvi.xlsx', index=False)